## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import json
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
file_location = os.path.join("..", "weather_database", "WeatherPy_database.csv")
city_data_df = pd.read_csv(file_location)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Haines Junction,CA,60.7522,-137.5108,35.55,60,100,1.14,overcast clouds
1,Mataura,NZ,-46.1927,168.8643,43.48,94,17,2.73,few clouds
2,Punta Arenas,CL,-53.1500,-70.9167,50.11,43,40,37.98,scattered clouds
3,Ushuaia,AR,-54.8000,-68.3000,49.66,43,75,21.85,broken clouds
4,Castro,BR,-24.7911,-50.0119,83.62,34,97,6.02,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,Castro,BR,-24.7911,-50.0119,83.62,34,97,6.02,overcast clouds
14,Corn Island,NI,12.1694,-83.0418,83.14,74,100,7.96,light rain
15,Altamira,BR,-3.2033,-52.2064,86.09,66,40,3.44,light rain
17,Lumeje,AO,-11.5500,20.7833,76.93,32,25,1.83,scattered clouds
18,Hilo,US,19.7297,-155.0900,79.29,78,75,0.00,light rain
21,Giyani,ZA,-23.3025,30.7187,83.46,25,53,4.63,broken clouds
29,Butaritari,KI,3.0707,172.7902,83.12,77,10,23.94,clear sky
31,Marsh Harbour,BS,26.5412,-77.0636,80.40,60,77,10.80,broken clouds
36,Kavaratti,IN,10.5669,72.6420,81.43,80,99,18.21,overcast clouds
37,Atuona,PF,-9.8000,-139.0333,77.61,76,21,19.80,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   214
Country                214
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean = preferred_cities_df.dropna()
preferred_cities_clean.count()

City                   214
Country                214
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Castro,BR,83.62,overcast clouds,-24.7911,-50.0119,
14,Corn Island,NI,83.14,light rain,12.1694,-83.0418,
15,Altamira,BR,86.09,light rain,-3.2033,-52.2064,
17,Lumeje,AO,76.93,scattered clouds,-11.5500,20.7833,
18,Hilo,US,79.29,light rain,19.7297,-155.0900,
21,Giyani,ZA,83.46,broken clouds,-23.3025,30.7187,
29,Butaritari,KI,83.12,clear sky,3.0707,172.7902,
31,Marsh Harbour,BS,80.40,broken clouds,26.5412,-77.0636,
36,Kavaratti,IN,81.43,overcast clouds,10.5669,72.6420,
37,Atuona,PF,77.61,light rain,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6d. Set up the base URL for the Google Directions API to get JSON data.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# print(json.dumps(hotels, indent = 2, sort_keys=True))

# 6b. Iterate through the hotel DataFrame.
for index, coord in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{coord['Lat']},{coord['Lng']}" 
    
    try:
        # 6e. Make request and retrieve the JSON data from the search.
        results = requests.get(base_url, params=params).json()
        
        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = results["results"][0]["name"]
    except:
        # Set the hotel name to null if no hotel is found
        hotel_df.loc[index, "Hotel Name"] = np.nan

hotel_df.head()        

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Castro,BR,83.62,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
14,Corn Island,NI,83.14,light rain,12.1694,-83.0418,Big Fish Guest House
15,Altamira,BR,86.09,light rain,-3.2033,-52.2064,Requinte Hotel
17,Lumeje,AO,76.93,scattered clouds,-11.5500,20.7833,NaN
18,Hilo,US,79.29,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Castro,BR,83.62,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
14,Corn Island,NI,83.14,light rain,12.1694,-83.0418,Big Fish Guest House
15,Altamira,BR,86.09,light rain,-3.2033,-52.2064,Requinte Hotel
18,Hilo,US,79.29,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
21,Giyani,ZA,83.46,broken clouds,-23.3025,30.7187,Kulani Country Lodge


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index=False)

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))